In [1]:
# Necessary imports

from sklearn.linear_model import LinearRegression
import pandas as pd
import cv2
import os
import io
import numpy as np
from PIL import Image
import numpy as np
import cv2
import matplotlib.pyplot as plt
import math
import torch
from torchvision.models.segmentation import deeplabv3_resnet101
from torchvision import transforms
import joblib

In [2]:
# Measurements file of subjects

df_measurements = pd.read_csv('train/measurements.csv')
df_measurements.head(5)

,subject_id,ankle,arm-length,bicep,calf,chest,forearm,height,hip,leg-length,shoulder-breadth,shoulder-to-crotch,thigh,waist,wrist
0,-494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg,24.343954,52.902378,33.294395,38.709511,102.526024,27.599094,177.610992,108.047134,81.736435,35.915691,67.891273,57.872795,92.286171,16.709131
1,-5OHO2b5mRDY1rvAf07sYYtekEEX7ViEGmvRr7l4tfc,22.001783,46.763233,28.288971,35.649963,99.253395,24.813375,164.379623,92.128685,75.127258,33.848778,60.541386,52.418320,87.307899,15.192038
2,-5iHTazqnmWBsnUY6ymoCw1kORMymRDaUla1eNOjgvI,26.565006,53.288597,30.800959,39.854950,102.391273,27.719795,183.718842,106.044121,84.055527,37.856457,72.004379,52.934086,96.596413,17.936598
3,-60o95b9oEE83BDV3GpsAVPA4TFvxuPPqtXEY5PApy8,24.779566,49.077751,34.613113,38.104267,111.164070,28.550409,169.059113,109.529465,74.310234,38.197800,68.536713,58.656673,103.395119,17.029860
4,-67TrwlJ3GSsqPeHaa2Nb0iQ-BKZfIx4ERysQPZ0-SY,20.788853,42.229557,24.527142,29.181356,93.627777,21.979038,146.516632,87.134102,67.269859,31.659197,52.697277,41.844944,74.400856,15.120061


In [3]:
# Metadata for subjects

df_info = pd.read_csv('train/hwg_metadata.csv')
df_info.head(5)

,subject_id,gender,height_cm,weight_kg
0,AhfyGaD1BKo0-u6PeQk2oI-Bw5l0XxpAqNroFuZuy0I,female,160.00,92.4
1,E8tDQ5IgdBwbjLIXu7GwcupEHYbi0eB18YNcsgsSbP8,female,175.75,102.8
2,59bc2767681516389ccf45f9,male,174.80,106.9
3,nRatvgqbYTN77typTgGm4bAurxzpvf5nYg2Mb3gdh8c,male,181.50,111.8
4,fkq68ww6bdyKT16FavI1x_G-u7YZ9SoT1GNMUTSJUek,female,161.60,93.0


In [4]:
# Subject to photo mapping

df_photo_map = pd.read_csv('train/subject_to_photo_map.csv')
df_photo_map.head(5)

,subject_id,photo_id
0,-494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg,e6f404ebda41ebe93573d3e219c88297
1,-494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg,39a4241cb892618a694747e224d57b9e
2,-494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg,a00a6c0c074efbd92c894cbc77023939
3,-494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg,7c99b3017f21f9797851e70de084bfcb
4,-494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg,3000cb26b2090db5a7d82926af8c6a83


In [5]:
# Merge Measurements with Metadata

df_subject_info_merged = pd.merge(df_measurements, df_info, on='subject_id', how='inner')
df_subject_info_merged.head(5)
df_photo_map = pd.merge(df_photo_map, df_subject_info_merged[['subject_id', 'height']], on='subject_id', how='inner')

In [6]:
# Method to return the image path of subjects of front view and left view

def return_image_path(photo_id):
    image_dir = 'train/mask'
    image_path_mask = os.path.join(image_dir, f'{photo_id}.png')
    
    
    image_dir = 'train/mask_left'
    image_path_maskleft = os.path.join(image_dir, f'{photo_id}.png')
    
    image = cv2.imread(image_path_maskleft)
    if image is None:
        print(f'Warning: Image {photo_id}.png not found')
    return [image_path_mask, image_path_maskleft]

In [7]:
#Extract features from imagges

def extract_silhouette_features(photo_id, subject_height):
    if '.png' in photo_id:
        image_ = [photo_id]
    else:
        image_ = return_image_path(photo_id=photo_id)
    features = {}
    loop = 0
    for image in image_:
        # Convert image to grayscale
        image = cv2.imread(image)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        if loop == 0:
            loop = 1
        # Thresholding to get binary silhouette
        ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
        
        # Find contours
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
           
        # Iterate through contours
        for contour in contours:
            # Calculate area and perimeter
            area = cv2.contourArea(contour)
            perimeter = cv2.arcLength(contour, True)
            
            # Calculate bounding box
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = float(w) / h if h != 0 else 0.0
            
            # Calculate solidity (area / convex hull area)
            hull = cv2.convexHull(contour)
            hull_area = cv2.contourArea(hull)
            solidity = float(area) / hull_area if hull_area != 0 else 0.0
            
            # Add features to dictionary
            scale = subject_height / h  # Use height of bounding box for scaling
            if loop == 2:
                features['area_left'] = area * (scale ** 2)
                features['perimeter_left'] = perimeter * scale
                features['aspect_ratio_left'] = aspect_ratio
                features['solidity_left'] = solidity
            else:
                features['area'] = area * (scale ** 2)
                features['perimeter'] = perimeter * scale
                features['aspect_ratio'] = aspect_ratio
                features['solidity'] = solidity
            
        loop = 2
    
    return features

In [8]:
# Extract image features and append 

df_photo_map['metrics'] = df_photo_map.apply(lambda row: extract_silhouette_features(row['photo_id'], row['height']), axis=1)
df_photo_map.head(10)

In [ ]:
# Extract each feature from the dict 

df_metrics = pd.json_normalize(df_photo_map['metrics'])

In [ ]:
# Map extracted features for each image wrt to subject

df_photo_map_extended = pd.concat([df_photo_map[['subject_id', 'photo_id']], df_metrics], axis=1)
df_photo_map_extended.head(5)

,subject_id,photo_id,area,perimeter,aspect_ratio,solidity,area_left,perimeter_left,aspect_ratio_left,solidity_left
0,-494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg,e6f404ebda41ebe93573d3e219c88297,5438.283665,848.828307,0.530201,0.487090,3351.131618,447.621411,0.177246,0.712320
1,-494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg,39a4241cb892618a694747e224d57b9e,5523.844534,855.200403,0.535161,0.489721,3336.161974,458.780808,0.182588,0.669973
2,-494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg,a00a6c0c074efbd92c894cbc77023939,5399.136663,860.225347,0.637850,0.422959,3394.749374,464.427354,0.182236,0.661618
3,-494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg,7c99b3017f21f9797851e70de084bfcb,5370.218622,855.052751,0.644737,0.416841,3302.340267,464.189041,0.185273,0.639856
4,-494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg,3000cb26b2090db5a7d82926af8c6a83,5497.382593,857.019574,0.606538,0.443443,3375.689709,464.819241,0.173759,0.676206


In [ ]:
# Drop column photid

df_photo_map_extended = df_photo_map_extended.drop(columns=['photo_id'])

In [ ]:
# Avg the metrics for each subject since some subjects have multiple images in dataset

df_final_one = df_photo_map_extended.groupby('subject_id').mean().reset_index()
df_final_one.head(5)

,subject_id,area,perimeter,aspect_ratio,solidity,area_left,perimeter_left,aspect_ratio_left,solidity_left
0,-494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg,5441.980663,854.525379,0.591337,0.451070,3355.960442,460.776183,0.179143,0.672697
1,-5OHO2b5mRDY1rvAf07sYYtekEEX7ViEGmvRr7l4tfc,4758.696002,772.641731,0.654762,0.428414,3202.230909,454.878165,0.215909,0.679564
2,-5iHTazqnmWBsnUY6ymoCw1kORMymRDaUla1eNOjgvI,5685.249656,885.516031,0.614127,0.432407,3510.647836,513.160955,0.206949,0.623788
3,-60o95b9oEE83BDV3GpsAVPA4TFvxuPPqtXEY5PApy8,5735.683907,826.203697,0.592995,0.483474,3362.804108,455.675578,0.188049,0.728562
4,-67TrwlJ3GSsqPeHaa2Nb0iQ-BKZfIx4ERysQPZ0-SY,3825.123732,689.014176,0.619385,0.468516,2608.767974,434.465633,0.219114,0.664346


In [ ]:
# Merging info df with finalone to extract info of subjects 
df_for_merge = df_subject_info_merged[['subject_id', 'weight_kg', 'height', 'wrist', 'waist', 'gender', 'hip']]

df = pd.merge(df_final_one, df_for_merge, on='subject_id', how='inner')

In [ ]:
# Convert categorical col gender to numerical

df['gender'] = df['gender'].apply(lambda x:1 if x =='male' else 0)

In [ ]:
df.head(5)

,subject_id,area,perimeter,aspect_ratio,solidity,area_left,perimeter_left,aspect_ratio_left,solidity_left,weight_kg,height,wrist,waist,gender,hip
0,-494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg,5441.980663,854.525379,0.591337,0.451070,3355.960442,460.776183,0.179143,0.672697,79.6,177.610992,16.709131,92.286171,1,108.047134
1,-5OHO2b5mRDY1rvAf07sYYtekEEX7ViEGmvRr7l4tfc,4758.696002,772.641731,0.654762,0.428414,3202.230909,454.878165,0.215909,0.679564,65.0,164.379623,15.192038,87.307899,0,92.128685
2,-5iHTazqnmWBsnUY6ymoCw1kORMymRDaUla1eNOjgvI,5685.249656,885.516031,0.614127,0.432407,3510.647836,513.160955,0.206949,0.623788,82.9,183.718842,17.936598,96.596413,1,106.044121
3,-60o95b9oEE83BDV3GpsAVPA4TFvxuPPqtXEY5PApy8,5735.683907,826.203697,0.592995,0.483474,3362.804108,455.675578,0.188049,0.728562,88.7,169.059113,17.029860,103.395119,1,109.529465
4,-67TrwlJ3GSsqPeHaa2Nb0iQ-BKZfIx4ERysQPZ0-SY,3825.123732,689.014176,0.619385,0.468516,2608.767974,434.465633,0.219114,0.664346,44.2,146.516632,15.120061,74.400856,0,87.134102


In [ ]:
# Normalize data

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Select the columns to normalize
columns_to_normalize = ['area','height', 'weight_kg','perimeter', 'aspect_ratio', 'solidity', 'area_left',
                         'perimeter_left', 'aspect_ratio_left', 'solidity_left']

# Fit and transform the data
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

In [ ]:
df.head(5)

,subject_id,area,perimeter,aspect_ratio,solidity,area_left,perimeter_left,aspect_ratio_left,solidity_left,weight_kg,height,wrist,waist,gender,hip
0,-494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg,0.203702,0.933347,0.593315,-1.061744,-0.115476,-0.271510,-0.249887,-0.314721,0.276159,0.585527,16.709131,92.286171,1,108.047134
1,-5OHO2b5mRDY1rvAf07sYYtekEEX7ViEGmvRr7l4tfc,-0.407777,-0.675325,1.236983,-1.487022,-0.189099,-0.438275,0.016917,-0.171355,-0.609718,-0.833141,15.192038,87.307899,0,92.128685
2,-5iHTazqnmWBsnUY6ymoCw1kORMymRDaUla1eNOjgvI,0.421406,1.542184,0.824603,-1.412060,-0.041395,1.209657,-0.048106,-1.335699,0.476392,1.240410,17.936598,96.596413,1,106.044121
3,-60o95b9oEE83BDV3GpsAVPA4TFvxuPPqtXEY5PApy8,0.466540,0.376944,0.610142,-0.453481,-0.112199,-0.415729,-0.185261,0.851470,0.828315,-0.331406,17.029860,103.395119,1,109.529465
4,-67TrwlJ3GSsqPeHaa2Nb0iQ-BKZfIx4ERysQPZ0-SY,-1.243241,-2.318257,0.877965,-0.734267,-0.473314,-1.015435,0.040176,-0.489040,-1.871790,-2.748410,15.120061,74.400856,0,87.134102


In [ ]:
# Model for wrist size calculation

model_wrist = LinearRegression()
model_wrist.fit(df.drop(columns=['subject_id', 'waist', 'wrist','hip']), df[['wrist']])
model_wrist_file = "models/wrist_linear_regression_model.pkl"
joblib.dump(model_wrist, model_wrist_file)

['models/wrist_linear_regression_model.pkl']

In [ ]:
# model_wrist_file = "models/wrist_linear_regression_model.pkl"
# model_wrist = joblib.load(model_wrist_file)

In [ ]:
# Model for waist size calculation

model_waist = LinearRegression()
model_waist.fit(df.drop(columns=['subject_id', 'waist', 'wrist','hip']), df[['waist']])
model_waist_file = "models/waist_linear_regression_model.pkl"
joblib.dump(model_waist, model_waist_file)


['models/waist_linear_regression_model.pkl']

In [ ]:
# model_waist_file = "models/waist_linear_regression_model.pkl"
# model_waist = joblib.load(model_waist_file)

In [ ]:
# Model for hip size calculation

model_hip = LinearRegression()
model_hip.fit(df.drop(columns=['subject_id', 'waist', 'wrist','hip']), df[['hip']])
model_hip_file = "models/hip_linear_regression_model.pkl"
joblib.dump(model_hip, model_hip_file)

['models/hip_linear_regression_model.pkl']

In [ ]:
# model_hip_file = "models/hip_linear_regression_model.pkl"
# model_hip = joblib.load(model_hip_file)

In [ ]:
# Method to extract features for inference images

def extract_silhouette_features_for_inference(image, subject_height):
    image_array = np.array(image)
    features ={}
    gray = cv2.cvtColor(image_array, cv2.COLOR_BGR2GRAY)
    
    # Thresholding to get binary silhouette
    ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    
    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
    # Iterate through contours
    for contour in contours:
        # Calculate area and perimeter
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        
        # Calculate bounding box
        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = float(w) / h if h != 0 else 0.0
        
        # Calculate solidity (area / convex hull area)
        hull = cv2.convexHull(contour)
        hull_area = cv2.contourArea(hull)
        solidity = float(area) / hull_area if hull_area != 0 else 0.0

        scale = subject_height / h  # Use height of bounding box for scaling
        features['area'] = area * (scale ** 2)
        features['perimeter'] = perimeter * scale
        features['aspect_ratio'] = aspect_ratio
        features['solidity'] = solidity

    return features
            

In [ ]:
# load prebuilt Resnet model 

def make_deeplab(device):
    deeplab = deeplabv3_resnet101(pretrained=True).to(device)
    deeplab.eval()
    return deeplab

device = torch.device("cpu")
deeplab = make_deeplab(device)

/Users/acehunter/PycharmProjects/BodyScan/.venv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/acehunter/PycharmProjects/BodyScan/.venv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# resize images

def preprocess(f_name):
    img_orig = cv2.imread(f_name, 1)
    k = min(1.0, 1024/max(img_orig.shape[0], img_orig.shape[1]))
    img = cv2.resize(img_orig, None, fx=k, fy=k, interpolation=cv2.INTER_LANCZOS4)
    return img

In [ ]:
# Generate masked images

deeplab_preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def apply_deeplab(deeplab, img, device):
    input_tensor = deeplab_preprocess(img)
    input_batch = input_tensor.unsqueeze(0)
    with torch.no_grad():
        output = deeplab(input_batch.to(device))['out'][0]
    output_predictions = output.argmax(0).cpu().numpy()
    return (output_predictions == 15)

mask_front = apply_deeplab(deeplab, preprocess("front-1.jpeg"), device)


In [ ]:
# Buff to store image temp

def create_buff(mask):
    buf = io.BytesIO()
    plt.imsave(buf, mask, cmap="gray", format="png")
    buf.seek(0)
    return buf

In [ ]:
# Access the image stored in buff

def access_stored_image(buffer):
    # Open the image from the BytesIO buffer
    image = Image.open(buffer)
    #image.show()  # Display the image (optional)
    return image

# Call the method with the stored buffer
stored_image = access_stored_image(create_buff(mask_front))

In [ ]:
# height and weight of subject

height = 174
weight = 99

In [ ]:
extract_silhouette_features_for_inference(stored_image, height)

{'area': 8662.94034318341,
 'perimeter': 766.6099433068588,
 'aspect_ratio': 0.4775967413441955,
 'solidity': 0.7764265916338164}

In [ ]:
mask_left = apply_deeplab(deeplab, preprocess("left-1.jpeg"), device)

In [ ]:
stored_image_left = access_stored_image(create_buff(mask_left))

In [ ]:
extract_silhouette_features_for_inference(stored_image_left, height)

{'area': 7125.006907939911,
 'perimeter': 536.6236205755267,
 'aspect_ratio': 0.46875,
 'solidity': 0.6878672066624836}

In [ ]:
dict1 = extract_silhouette_features_for_inference(stored_image, height)
dict2 = extract_silhouette_features_for_inference(stored_image_left, height)

In [ ]:
combined_dict = {
    'area': dict1['area'],
    'perimeter': dict1['perimeter'],
    'aspect_ratio': dict1['aspect_ratio'],
    'solidity': dict1['solidity'],
    'area_left': dict2['area'],
    'perimeter_left': dict2['perimeter'],
    'aspect_ratio_left': dict2['aspect_ratio'],
    'solidity_left': dict2['solidity']
}

# Convert the combined dictionary to a DataFrame
df_inference = pd.DataFrame([combined_dict])
df_inference.head(5)

,area,perimeter,aspect_ratio,solidity,area_left,perimeter_left,aspect_ratio_left,solidity_left
0,8662.940343,766.609943,0.477597,0.776427,7125.006908,536.623621,0.46875,0.687867


In [ ]:
# merge with image metrics

df_inference['weight_kg'] = 99
df_inference['height'] = 174
df_inference['gender'] = 1

In [ ]:
df_inference.head(5)

,area,perimeter,aspect_ratio,solidity,area_left,perimeter_left,aspect_ratio_left,solidity_left,weight_kg,height,gender
0,8662.940343,766.609943,0.477597,0.776427,7125.006908,536.623621,0.46875,0.687867,100,183,1


In [ ]:
# normalize wrt test data

mean = scaler.mean_
scale = scaler.scale_

In [ ]:
df_inference['area'] = (df_inference['area'] - mean[0])/scale[0]
df_inference['height'] = (df_inference['height'] - mean[1])/scale[1]
df_inference['weight_kg'] = (df_inference['weight_kg'] - mean[2])/scale[2]
df_inference['perimeter'] = (df_inference['perimeter'] - mean[3])/scale[3]
df_inference['aspect_ratio'] = (df_inference['aspect_ratio'] - mean[4])/scale[4]
df_inference['solidity'] = (df_inference['solidity'] - mean[5])/scale[5]
df_inference['area_left'] = (df_inference['area_left'] - mean[6])/scale[6]
df_inference['perimeter_left'] = (df_inference['perimeter_left'] - mean[7])/scale[7]
df_inference['aspect_ratio_left'] = (df_inference['aspect_ratio_left'] - mean[8])/scale[8]
df_inference['solidity_left'] = (df_inference['solidity_left'] - mean[9])/scale[9]

In [ ]:
df_inference.head(5)

,area,perimeter,aspect_ratio,solidity,area_left,perimeter_left,aspect_ratio_left,solidity_left,weight_kg,height,gender
0,3.086174,-0.793824,-0.560973,5.045529,1.689556,1.873059,1.851743,0.001965,1.51396,1.163336,1


In [ ]:
# Predict Wrist 

wrist_circumfurance = model_wrist.predict(df_inference)

In [ ]:
# Predict Waist

waist_circumfurance = model_waist.predict(df_inference)

In [ ]:
# Predict hip

hip_circumfurance = model_hip.predict(df_inference)

In [ ]:
# calc neck cir from wrist cir

neck_circumference= 2.64+1.94 * wrist_circumfurance

In [ ]:
print(f'Neck:{neck_circumference}')
print(f'Waist:{waist_circumfurance}')
print(f'Hip:{hip_circumfurance}')

Neck:[[38.90770216]]
Waist:[[107.01397122]]
Hip:[[116.07184511]]


In [ ]:
def calculate_final_metrics(sex, neck_circumference, waist_circumference, hip_circumference, height, weight):

    def calculate_bfp(sex, neck_circumference, waist_circumference, hip_circumference, height):
        """
        Calculate Body Fat Percentage using the Navy Body Fat formula.
        """
        if sex.lower() == 'male':
            bfp = 86.010 * math.log10(waist_circumference - neck_circumference) - 70.041 * math.log10(height) + 36.76
        elif sex.lower() == 'female':
            bfp = 163.205 * math.log10(waist_circumference + hip_circumference - neck_circumference) - 97.684 * math.log10(height) - 78.387
        else:
            raise ValueError("Sex must be 'male' or 'female'")
        return bfp

    def classify_fat_types(weight, bfp):
        """
        Classify fat types into essential, beneficial, and unbeneficial fats.
        """
        fm = weight * (bfp / 100)
        essential_fat = 0.05 * weight  # 5% of total body weight for essential fat
        beneficial_fat = 0.15 * weight  # 15% of total body weight for beneficial fat
        unbeneficial_fat = fm - (essential_fat + beneficial_fat)
        return essential_fat, beneficial_fat, unbeneficial_fat

    def calculate_lean_mass(weight, fm):
        """
        Calculate Lean Mass.
        """
        return weight - fm

    def calculate_indices(lean_mass, fm, height):
        """
        Calculate Lean Mass Index (LMI) and Fat Mass Index (FMI).
        """
        height_m = height / 100  # convert height to meters
        lmi = lean_mass / (height_m ** 2)
        fmi = fm / (height_m ** 2)
        return lmi, fmi

    def calculate_rmr(lean_mass):
        """
        Calculate Resting Metabolic Rate (RMR) using the Katch-McArdle formula.
        """
        return 370 + (21.6 * lean_mass)

    # Calculations
    bfp = calculate_bfp(sex, neck_circumference, waist_circumference, hip_circumference, height)
    fm = weight * (bfp / 100)
    essential_fat, beneficial_fat, unbeneficial_fat = classify_fat_types(weight, bfp)
    lean_mass = calculate_lean_mass(weight, fm)
    lmi, fmi = calculate_indices(lean_mass, fm, height)
    rmr = calculate_rmr(lean_mass)

    # Print Results
    print(f"Body Fat Percentage (BFP): {bfp:.2f}%")
    print(f"Essential Fat: {essential_fat:.2f} kg")
    print(f"Beneficial Fat: {beneficial_fat:.2f} kg")
    print(f"Unbeneficial Fat: {unbeneficial_fat:.2f} kg")
    print(f"Lean Mass: {lean_mass:.2f} kg")
    print(f"Lean Mass Index (LMI): {lmi:.2f} kg/m^2")
    print(f"Fat Mass Index (FMI): {fmi:.2f} kg/m^2")
    print(f"Resting Metabolic Rate (RMR): {rmr:.2f} kcal/day")


In [ ]:
calculate_final_metrics('male', neck_circumference, waist_circumfurance, hip_circumfurance, height, weight)

Body Fat Percentage (BFP): 35.97%
Essential Fat: 5.00 kg
Beneficial Fat: 15.00 kg
Unbeneficial Fat: 15.97 kg
Lean Mass: 64.03 kg
Lean Mass Index (LMI): 19.12 kg/m^2
Fat Mass Index (FMI): 10.74 kg/m^2
Resting Metabolic Rate (RMR): 1753.09 kcal/day


/var/folders/02/vgb90thx747_z8w7ph05k5tw0000gn/T/ipykernel_32714/2588190403.py:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  bfp = 86.010 * math.log10(waist_circumference - neck_circumference) - 70.041 * math.log10(height) + 36.76
